In [360]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [368]:
ling_features = pd.read_csv("linguistic_features/linguistic_feature_set.csv", index_col = 0)
display(ling_features)
ling_features_array = np.array(ling_features)
feature_data = np.array(ling_features)[:, 1:]
coherence_data = ling_features_array[:, 0]
print(feature_data)
print(coherence_data)
display(ling_features)

,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,POS:ADJ,POS:ADV,POS:CCONJ,POS:SCONJ,POS:DET,...,flesch_reading_ease,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,3,0.031521,0.007830,-0.282531,0.624773,-0.383468,1.048965,-0.641477,0.204631,0.549315,...,0.497214,0.538403,0.276915,-0.555212,0.404909,-0.498805,0.284006,-0.891422,-0.328190,-0.997350
1,4,0.371692,-0.719002,0.410407,0.537476,-0.445036,1.019417,-0.209887,-0.342357,-0.023866,...,0.225059,0.915415,1.115458,0.105653,0.094349,0.031449,0.030278,0.006135,-0.162578,-0.006094
2,4,4.083993,-0.042397,1.066779,-0.827151,0.010404,0.206455,-0.471540,0.474520,0.032879,...,-0.881179,-0.976627,-1.100609,0.494636,0.749187,-0.365937,0.362749,0.468033,0.429886,0.185372
3,4,4.483324,0.547678,0.118736,-0.464441,0.744873,0.769418,0.075642,-0.990419,0.114524,...,-0.092167,-0.958557,-1.067427,-1.367487,-0.201459,-0.852027,-1.165053,-0.031574,0.343783,-0.395351
4,5,-0.375206,0.816435,-0.500332,0.251526,-0.846909,-0.636591,1.030509,0.844017,0.594168,...,-0.522522,0.726909,0.736075,0.546963,1.288115,-0.092700,1.140528,-1.222055,-0.195101,-0.157682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.001940,0.535744,-0.683940,0.113069,1.280264,0.859907,1.765604,1.183605,-1.165806,...,-0.804405,0.204892,0.077405,-0.256370,-0.470404,0.053868,1.001978,-0.696857,0.201319,0.785644
1535,4,-0.811512,-0.677771,-0.683940,-0.007805,0.838726,3.087055,1.158446,1.105922,-0.720243,...,-0.346858,0.726909,0.140408,-1.458316,-1.879093,0.288023,0.270291,-1.206942,-0.994663,-0.272723
1536,3,-0.774537,1.634691,-0.683940,-0.748751,0.729269,-0.199267,0.427377,1.012384,1.023363,...,-0.817825,-0.906810,-0.327616,-1.041040,-0.784908,-1.078681,0.151473,0.936754,-0.256688,0.106877
1537,5,-0.552686,-0.071569,0.185778,0.282236,-0.865056,0.109646,0.232586,-0.117447,0.070380,...,-0.278560,0.349897,0.959450,-0.292094,0.353314,0.182260,-1.074511,-0.673048,-0.482177,-0.581193


[[ 0.03152056  0.00782976 -0.28253133 ... -0.89142235 -0.32818965
  -0.99735041]
 [ 0.37169159 -0.7190017   0.41040745 ...  0.00613471 -0.16257807
  -0.00609352]
 [ 4.08399281 -0.04239675  1.06677906 ...  0.46803307  0.42988572
   0.1853719 ]
 ...
 [-0.77453688  1.63469088 -0.68393959 ...  0.93675361 -0.25668824
   0.10687707]
 [-0.55268621 -0.07156884  0.18577832 ... -0.6730476  -0.48217667
  -0.58119321]
 [-0.85588212  2.0921009   1.00224819 ... -0.02454079  1.20201928
   1.47959592]]
[3. 4. 4. ... 3. 5. 5.]


,Coherence,n_words,POS:NOUN,POS:PROPN,POS:PRON,POS:ADJ,POS:ADV,POS:CCONJ,POS:SCONJ,POS:DET,...,flesch_reading_ease,coref_local,coref_global,n_constituents,n_noun_phrases,n_verb_phrases,n_sub_clauses,n_prep_phrases,mean_np_length,mean_vp_length
0,3,0.031521,0.007830,-0.282531,0.624773,-0.383468,1.048965,-0.641477,0.204631,0.549315,...,0.497214,0.538403,0.276915,-0.555212,0.404909,-0.498805,0.284006,-0.891422,-0.328190,-0.997350
1,4,0.371692,-0.719002,0.410407,0.537476,-0.445036,1.019417,-0.209887,-0.342357,-0.023866,...,0.225059,0.915415,1.115458,0.105653,0.094349,0.031449,0.030278,0.006135,-0.162578,-0.006094
2,4,4.083993,-0.042397,1.066779,-0.827151,0.010404,0.206455,-0.471540,0.474520,0.032879,...,-0.881179,-0.976627,-1.100609,0.494636,0.749187,-0.365937,0.362749,0.468033,0.429886,0.185372
3,4,4.483324,0.547678,0.118736,-0.464441,0.744873,0.769418,0.075642,-0.990419,0.114524,...,-0.092167,-0.958557,-1.067427,-1.367487,-0.201459,-0.852027,-1.165053,-0.031574,0.343783,-0.395351
4,5,-0.375206,0.816435,-0.500332,0.251526,-0.846909,-0.636591,1.030509,0.844017,0.594168,...,-0.522522,0.726909,0.736075,0.546963,1.288115,-0.092700,1.140528,-1.222055,-0.195101,-0.157682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,3,0.001940,0.535744,-0.683940,0.113069,1.280264,0.859907,1.765604,1.183605,-1.165806,...,-0.804405,0.204892,0.077405,-0.256370,-0.470404,0.053868,1.001978,-0.696857,0.201319,0.785644
1535,4,-0.811512,-0.677771,-0.683940,-0.007805,0.838726,3.087055,1.158446,1.105922,-0.720243,...,-0.346858,0.726909,0.140408,-1.458316,-1.879093,0.288023,0.270291,-1.206942,-0.994663,-0.272723
1536,3,-0.774537,1.634691,-0.683940,-0.748751,0.729269,-0.199267,0.427377,1.012384,1.023363,...,-0.817825,-0.906810,-0.327616,-1.041040,-0.784908,-1.078681,0.151473,0.936754,-0.256688,0.106877
1537,5,-0.552686,-0.071569,0.185778,0.282236,-0.865056,0.109646,0.232586,-0.117447,0.070380,...,-0.278560,0.349897,0.959450,-0.292094,0.353314,0.182260,-1.074511,-0.673048,-0.482177,-0.581193


In [377]:
train_data, test_data, train_coherence, test_coherence = train_test_split(feature_data, coherence_data, test_size = 0.33, random_state = 42)
mlp = MLPClassifier(random_state = 42, max_iter = 1000)
mlp.fit(train_data, train_coherence)

/Users/emicatx/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=1000, random_state=42)

In [386]:
mlp.predict(test_data)
mlp.score(test_data, test_coherence)

0.44861660079051385

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

accuracy = 0
for i in range(100):
    train_data, test_data, train_coherence, test_coherence = train_test_split(feature_data, coherence_data, test_size = 0.33, random_state = 42)

    rf_classifier = RandomForestClassifier(n_estimators=100)
    rf_classifier.fit(train_data, train_coherence)

    pred_coherence = rf_classifier.predict(test_data)
    accuracy = accuracy_score(test_coherence, pred_coherence)

    accuracy += accuracy

mean_accuracy = accuracy / 100
print(mean_accuracy)